In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
# os.chdir("/content/drive/MyDrive")
# %cd MELDData
os.getcwd()

'/content'

In [ ]:
!wget http://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz

--2025-08-29 08:18:14--  http://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz
Resolving web.eecs.umich.edu (web.eecs.umich.edu)... 141.212.113.214
Connecting to web.eecs.umich.edu (web.eecs.umich.edu)|141.212.113.214|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10878146150 (10G) [application/x-gzip]
Saving to: ‘MELD.Raw.tar.gz’

MELD.Raw.tar.gz     100%[===================>]  10.13G  10.4MB/s    in 16m 20s 

2025-08-29 08:34:33 (10.6 MB/s) - ‘MELD.Raw.tar.gz’ saved [10878146150/10878146150]



In [ ]:
## -x: This option signifies "extract."
## -f: This option specifies that the next argument will be the name of the archive file.
## -v: To view the list of files being extracted during the process
## The tar command is intelligent enough to automatically detect and handle various
##compression formats without needing specific decompression flags for most modern versions.
## if you encounter issues or are using an older tar version,
#you might need to explicitly include the decompression flag
## -z: flag indicates gzip decompression
!tar -xvzf MELD.Raw.tar.gz ## just copy paste do not take so hard on yourself it can be any name


MELD.Raw/
MELD.Raw/train.tar.gz
MELD.Raw/dev.tar.gz
MELD.Raw/test_sent_emo.csv
MELD.Raw/._train_splits
MELD.Raw/dev_sent_emo.csv
MELD.Raw/README.txt
MELD.Raw/test.tar.gz


In [ ]:
# # -c :  extract the contents into the MELD.Raw/ folder uppercase
# !tar -xvzf MELD.Raw/train.tar.gz -C MELD.Raw/
# !tar -xvzf MELD.Raw/dev.tar.gz -C MELD.Raw/
!tar -xvzf MELD.Raw/test.tar.gz -C MELD.Raw/


In [ ]:
## explore the folder structure
# List contents of MELD.Raw directory
# !ls MELD.Raw

# # List contents of train_splits folder
# !ls MELD.Raw/train_splits

# # List contents of dev_splits folder
# !ls MELD.Raw/dev_splits_complete

# # List contents of test_splits folder
# !ls MELD.Raw/output_repeated_splits_test


In [ ]:
import pandas as pd

# load emotion_labeled training data
train_df = pd.read_csv("/content/MELD.Raw/train_sent_emo.csv")
dev_df = pd.read_csv("/content/MELD.Raw/dev_sent_emo.csv")
test_df = pd.read_csv("MELD.Raw/test_sent_emo.csv")


# Display shape and first few rows
print("Train shape:", train_df.shape) # (9989, 11)
print("Dev shape:", dev_df.shape) # (1109, 11)
print("Test shape:", test_df.shape) # (2610, 11)

train_df.head()
# Check column names and data types



Train shape: (9989, 11)
Dev shape: (1109, 11)


,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [ ]:
test_df = pd.read_csv("MELD.Raw/test_sent_emo.csv")
print("Test shape:", test_df.shape) # (2610, 11)


In [ ]:
print("train_df_type:", train_df.dtypes)

# List unique emotion and sentiment categories
print("Emotion categories:", train_df['Emotion'].unique())
print("Sentiment categories:", train_df['Sentiment'].unique())

# Count distribution of emotions
emotion_counts = train_df['Emotion'].value_counts()
print("Emotion distribution:\n", emotion_counts)

# Count distribution of sentiments
sentiment_counts = train_df['Sentiment'].value_counts()
print("Sentiment distribution:\n", sentiment_counts)

train_df_type: Sr No.           int64
Utterance       object
Speaker         object
Emotion         object
Sentiment       object
Dialogue_ID      int64
Utterance_ID     int64
Season           int64
Episode          int64
StartTime       object
EndTime         object
dtype: object
Emotion categories: ['neutral' 'surprise' 'fear' 'sadness' 'joy' 'disgust' 'anger']
Sentiment categories: ['neutral' 'positive' 'negative']
Emotion distribution:
 Emotion
neutral     4710
joy         1743
surprise    1205
anger       1109
sadness      683
disgust      271
fear         268
Name: count, dtype: int64
Sentiment distribution:
 Sentiment
neutral     4710
negative    2945
positive    2334
Name: count, dtype: int64


In [ ]:
"""  Text Preprocessing

The text data in MELD is mostly clean, but for NLP tasks, you’ll likely want to:

    Lowercase all text for consistency

    Remove punctuation or special characters (if not needed for emotion cues)

    Tokenize sentences into words or subwords

    Handle contractions (e.g., "don't" → "do not") if using rule-based models

    Remove stopwords (optional, depending on your model)

    Convert to embeddings using BERT, RoBERTa, or GloVe

🔊 Audio Preprocessing

The .wav files are raw audio clips. You’ll need to:

    Extract features like MFCCs, spectrograms, or use pretrained models like Wav2Vec

    Normalize volume levels if needed

    Trim silence or pad clips to a fixed length

    Align audio with text using timestamps (already provided in MELD)

🎭 Label Preprocessing

    Convert categorical labels (Emotion, Sentiment) to numerical format

    Balance classes if you're training a classifier (some emotions are underrepresented)

    One-hot encode or use label encoding depending on your model

🧩 Multimodal Alignment

If you're doing multimodal fusion (text + audio + visual), you’ll need to:

    Synchronize modalities using timestamps

    Handle missing data (some utterances may lack audio or visual)

    Standardize feature dimensions across modalities
    """

'  Text Preprocessing\n\nThe text data in MELD is mostly clean, but for NLP tasks, you’ll likely want to:\n\n    Lowercase all text for consistency\n\n    Remove punctuation or special characters (if not needed for emotion cues)\n\n    Tokenize sentences into words or subwords\n\n    Handle contractions (e.g., "don\'t" → "do not") if using rule-based models\n\n    Remove stopwords (optional, depending on your model)\n\n    Convert to embeddings using BERT, RoBERTa, or GloVe\n\n🔊 Audio Preprocessing\n\nThe .wav files are raw audio clips. You’ll need to:\n\n    Extract features like MFCCs, spectrograms, or use pretrained models like Wav2Vec\n\n    Normalize volume levels if needed\n\n    Trim silence or pad clips to a fixed length\n\n    Align audio with text using timestamps (already provided in MELD)\n\n🎭 Label Preprocessing\n\n    Convert categorical labels (Emotion, Sentiment) to numerical format\n\n    Balance classes if you\'re training a classifier (some emotions are underrepresen

In [ ]:
# import gdown

# url = 'https://drive.google.com/file/d/1RjrYSMpXxg_6r_nUQaysaPyMsldLpMcb/view?usp=sharing'
# output = 'meld.tar'
# gdown.download(url, output, quiet=False)

In [ ]:
#!tar -xvf meld.tar

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
# !file meld.tar ##Ah, that explains the extraction error!
## The file you downloaded—meld.tar—is actually an HTML document, not a real .tar

meld.tar: HTML document, ASCII text, with very long lines (39274)


In [ ]:
!pip install gdown
!gdown --id 1RjrYSMpXxg_6r_nUQaysaPyMsldLpMcb --fuzzy --no-cookies

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1RjrYSMpXxg_6r_nUQaysaPyMsldLpMcb
From (redirected): https://drive.google.com/uc?id=1RjrYSMpXxg_6r_nUQaysaPyMsldLpMcb&confirm=t&uuid=282984b7-261d-4586-8611-dfda6d04ed14
To: /content/meld.tar
100% 2.29G/2.29G [00:32<00:00, 71.0MB/s]


In [ ]:
!file meld.tar ## it is correct now
!tar -xvf meld.tar

meld.tar: POSIX tar archive (GNU)
dev.pkl
test.pkl
train.pkl
embedding.p


In [ ]:
import pickle

with open('train.pkl', 'rb') as f:
    train_data = pickle.load(f)

# Check type and structure
print(type(train_data))  # likely list or dict
# sample = train_data[0]   # or train_data['some_id'] if it's a dict

# # Print keys and shapes
# for key in sample:
#     print(f"{key}: type={type(sample[key])}, shape={getattr(sample[key], 'shape', 'N/A')}")


<class 'dict'>


In [ ]:
# Print the number of samples
print("Number of samples:", len(train_data))

# Print the first key
first_key = list(train_data.keys())[0]
print("First key:", first_key)

# Print the structure of one sample
sample = train_data[first_key]
for k, v in sample.items():
    print(f"{k}: type={type(v)}, shape={getattr(v, 'shape', 'N/A')}")

Number of samples: 9988
First key: 0_0
token_ids: type=<class 'list'>, shape=N/A
audio_features: type=<class 'numpy.ndarray'>, shape=(19, 32)
video_features: type=<class 'numpy.ndarray'>, shape=(19, 2048)
label: type=<class 'int'>, shape=N/A


In [ ]:
'''
Number of samples: 9988 You have 9,988 data
 points—likely utterances or dialogue turns
 if you're working with something like MELD or another multimodal emotion dataset.

First key: 0_0 This is probably the ID for the first sample.
It might represent something like conversationID_utteranceID.
'''

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12, 4))
# plt.plot(sample['video_features'])
# plt.title('Video Feature Vector')
# plt.xlabel('Time Frame')
# plt.ylabel('Feature Value')
# plt.show()

In [ ]:
# ## this visualization can be messy so i recommend to :
# plt.plot(sample['video_features'][:, 0])
# plt.title('Video Feature - Dimension 0')


In [ ]:
# plt.imshow(sample['video_features'].T, aspect='auto', origin='lower', cmap='plasma')
# plt.title('Video Feature Map')
# plt.xlabel('Time Frame')
# plt.ylabel('Feature Dimension')
# plt.colorbar()

In [ ]:
'''
token_ids are not features yet. They are just numerical representations of words or subwords from the original utterance in meld.raw.
token_ids	Integer IDs for each word/subword	Input to text encoder
Text encoder	Model that converts IDs to feature vectors	Produces semantic text features
Raw text	Original utterance from MELD	Used for alignment and debugging
'''

'\ntoken_ids are not features yet. They are just numerical representations of words or subwords from the original utterance in meld.raw.\ntoken_ids\tInteger IDs for each word/subword\tInput to text encoder\nText encoder\tModel that converts IDs to feature vectors\tProduces semantic text features\nRaw text\tOriginal utterance from MELD\tUsed for alignment and debugging\n'

In [ ]:
# raw_lookup = {}
# for row in meld.Raw:  # or use meld_raw.iterrows() if it's a DataFrame
#     key = f"{row['Dialogue_ID']}_{row['Utterance_ID']}"
#     raw_lookup[key] = row
## we have Meld.Raw as dataframes

### build look up table for align with extracted multimodal features so you can train a model from scratch.

In [ ]:
train_df['key'] = train_df['Dialogue_ID'].astype(str) + '_' + train_df['Utterance_ID'].astype(str)

In [ ]:
raw_lookup = {
    row['key']: row
    for _, row in train_df.iterrows()
}

## compact and powerful way to build a lookup dictionary from your train_df DataFrame

In [ ]:
print(raw_lookup['0_0'])

Sr No.                                                          1
Utterance       also I was the point person on my companys tr...
Speaker                                                  Chandler
Emotion                                                   neutral
Sentiment                                                 neutral
Dialogue_ID                                                     0
Utterance_ID                                                    0
Season                                                          8
Episode                                                        21
StartTime                                            00:16:16,059
EndTime                                              00:16:21,731
key                                                           0_0
Name: 0, dtype: object


In [ ]:
'''
Imagine you have a spreadsheet with thousands of rows. Each row has:

    A conversation ID (Dialogue_ID)

    A line number (Utterance_ID)

    A season and episode

    A speaker and their line

You create a new column called key that says "DialogueID_UtteranceID"—like "12_4".

Now you build a dictionary where each key is that "12_4" and the value is the full row. That’s all this code is doing.

So No Mixing Happens

    Each row is stored under its own unique key.

    No rows are “combined” or “concatenated” across seasons.

    You can still access season/episode info from the row itself.'''

'\nSo No Mixing Happens\n\n    Each row is stored under its own unique key.\n\n    No rows are “combined” or “concatenated” across seasons.\n\n    You can still access season/episode info from the row itself.'

In [ ]:
'''
✅ Loaded and explored the MELD metadata
✅ Downloaded and unpacked the preprocessed multimodal features (train.pkl)
✅ Created a key to align metadata with feature samples
✅ Built a raw_lookup dictionary for fast access to metadata by key

Now you're ready to combine the metadata (train_df) with the extracted features (train_data) into a unified dataset class for training your fusion model '''

In [ ]:
import torch
from torch.utils.data import Dataset

class MELDFusionDataset(Dataset):
    def __init__(self, features_dict, raw_lookup, max_text_len=50, pad_token_id=0):
        self.keys = list(features_dict.keys())
        self.features = features_dict
        self.raw_lookup = raw_lookup
        self.max_text_len = max_text_len
        self.pad_token_id = pad_token_id

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        feat = self.features[key]
        raw = self.raw_lookup.get(key, None)

        # Pad or truncate token_ids
        token_ids = feat['token_ids']
        if len(token_ids) < self.max_text_len:
            token_ids += [self.pad_token_id] * (self.max_text_len - len(token_ids))
        else:
            token_ids = token_ids[:self.max_text_len]
        token_ids = torch.tensor(token_ids, dtype=torch.long)

        # Audio and video features
        audio = torch.tensor(feat['audio_features'], dtype=torch.float)
        video = torch.tensor(feat['video_features'], dtype=torch.float)

        # Label
        label = torch.tensor(feat['label'], dtype=torch.long)

        # Optional: include raw metadata
        return {
            'key': key,
            'token_ids': token_ids,
            'audio': audio,
            'video': video,
            'label': label,
            'utterance': raw['Utterance'] if raw is not None else None,
            'emotion': raw['Emotion'] if raw is not None else None,
            'speaker': raw['Speaker'] if raw is not None else None
        }
dataset = MELDFusionDataset(train_data, raw_lookup)
sample = dataset[0]

print("Text shape:", sample['token_ids'].shape)
print("Audio shape:", sample['audio'].shape)
print("Video shape:", sample['video'].shape)
print("Label:", sample['label'])
print("Utterance:", sample['utterance'])
print("Emotion:", sample['emotion'])


Text shape: torch.Size([50])
Audio shape: torch.Size([19, 32])
Video shape: torch.Size([19, 2048])
Label: tensor(0)
Utterance: also I was the point person on my companys transition from the KL-5 to GR-6 system.
Emotion: neutral


In [ ]:
'''
Downloaded and extracted the MELD dataset

Loaded and explored the emotion-labeled CSV files

Pulled in preprocessed features from a pickle file

Created a lookup dictionary for raw metadata

Defined a custom PyTorch Dataset class to fuse modalities '''

In [ ]:
## But here’s the issue: raw is a pandas Series (a row from your DataFrame), and Python doesn’t know how to evaluate if raw: because it’s not a single boolean—it’s a whole row of values.

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

def meld_collate_fn(batch):
    keys = [item['key'] for item in batch]
    token_ids = torch.stack([item['token_ids'] for item in batch])
    labels = torch.stack([item['label'] for item in batch])

    # Pad audio and video sequences
    audio_seqs = [item['audio'] for item in batch]
    video_seqs = [item['video'] for item in batch]

    padded_audio = pad_sequence(audio_seqs, batch_first=True)  # shape: [B, T, F]
    padded_video = pad_sequence(video_seqs, batch_first=True)  # shape: [B, T, F]

    # Optional metadata
    utterances = [item['utterance'] for item in batch]
    emotions = [item['emotion'] for item in batch]
    speakers = [item['speaker'] for item in batch]

    return {
        'keys': keys,
        'token_ids': token_ids,
        'audio': padded_audio,
        'video': padded_video,
        'labels': labels,
        'utterances': utterances,
        'emotions': emotions,
        'speakers': speakers
    }
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=meld_collate_fn
)


In [ ]:
batch = next(iter(train_loader))

print("Batch token_ids shape:", batch['token_ids'].shape)  # [B, T]
print("Batch audio shape:", batch['audio'].shape)          # [B, T, F]
print("Batch video shape:", batch['video'].shape)          # [B, T, F]
print("Batch labels:", batch['labels'])                    # [B]

print("\nSample utterances:")
for utt, emo in zip(batch['utterances'], batch['emotions']):
    print(f"Emotion: {emo} | Utterance: {utt}")


Batch token_ids shape: torch.Size([4, 50])
Batch audio shape: torch.Size([4, 15, 32])
Batch video shape: torch.Size([4, 15, 2048])
Batch labels: tensor([3, 4, 0, 4])

Sample utterances:
Emotion: sadness | Utterance: Umm, well I sorta have some bad news, can I come in?
Emotion: joy | Utterance: I am going to take you out to dinner tonight.
Emotion: neutral | Utterance: I got it. Uh, Joey, women don't have Adam's apples.
Emotion: joy | Utterance: Great!


In [ ]:
'''
Awesome! Let’s build a clean pipeline to handle batches from your MELDFusionDataset. We’ll create:

    A custom collate function to pad variable-length audio/video.

    A DataLoader to batch your data.

    A quick visualization of a batch to inspect shapes and contents.'''

'\nAwesome! Let’s build a clean pipeline to handle batches from your MELDFusionDataset. We’ll create:\n\n    A custom collate function to pad variable-length audio/video.\n\n    A DataLoader to batch your data.\n\n    A quick visualization of a batch to inspect shapes and contents.'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FusionEmotionModel(nn.Module):
    def __init__(self, text_dim, audio_dim, video_dim, hidden_dim, num_classes):
        super().__init__()

        # Temporal modeling for audio/video
        self.audio_rnn = nn.GRU(audio_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.video_rnn = nn.GRU(video_dim, hidden_dim, batch_first=True, bidirectional=True)

        # Text projection
        self.text_fc = nn.Sequential(
            nn.Linear(text_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Attention layers
        self.attn_audio = nn.Linear(hidden_dim * 2, 1)
        self.attn_video = nn.Linear(hidden_dim * 2, 1)

        # # Final fusion
        # self.classifier = nn.Sequential(
        #     nn.Linear(hidden_dim * 4, hidden_dim),
        #     nn.ReLU(),
        #     nn.Dropout(0.3),
        #     nn.Linear(hidden_dim, num_classes)

        self.classifier = nn.Sequential(
            nn.Linear(640, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes))

    def attention_pooling(self, rnn_out, attn_layer):
        weights = F.softmax(attn_layer(rnn_out), dim=1)  # [B, T, 1]
        pooled = torch.sum(weights * rnn_out, dim=1)     # [B, hidden]
        return pooled

    def forward(self, text_embed, audio_seq, video_seq):
        # Text
        text_feat = self.text_fc(text_embed)  # [B, hidden]

        # Audio
        audio_out, _ = self.audio_rnn(audio_seq)  # [B, T, 2*hidden]
        audio_feat = self.attention_pooling(audio_out, self.attn_audio)

        # Video
        video_out, _ = self.video_rnn(video_seq)  # [B, T, 2*hidden]
        video_feat = self.attention_pooling(video_out, self.attn_video)

        # Fusion
        fused = torch.cat([text_feat, audio_feat, video_feat], dim=1)
        return self.classifier(fused)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score


from sklearn.metrics import classification_report

def detailed_report(preds, labels, emotion_labels):
    preds = preds.argmax(dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    print(classification_report(labels, preds, target_names=emotion_labels))


def evaluate_metrics(preds, labels):
    preds = preds.argmax(dim=1).cpu().numpy()
    labels = labels.cpu().numpy()

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')  # or 'macro' for equal class weight
    return acc, f1
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModel.from_pretrained("bert-base-uncased")

def get_text_embedding(token_ids):
    with torch.no_grad():
        output = bert(token_ids)
        return output.last_hidden_state.mean(dim=1)  # [B, hidden_size]


In [ ]:
model = FusionEmotionModel(
    text_dim=768,     # BERT hidden size
    audio_dim=32,     # your audio feature size
    video_dim=2048,   # your video feature size
    hidden_dim=128,   # can be tuned
    num_classes=7     # adjust based on your emotion labels
)
model = FusionEmotionModel(
    text_dim=768,     # BERT hidden size
    audio_dim=32,     # your audio feature size
    video_dim=2048,   # your video feature size
    hidden_dim=128,   # can be tuned
    num_classes=7     # adjust based on your emotion labels
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

token_ids = batch['token_ids'].to(device)
audio = batch['audio'].to(device)
video = batch['video'].to(device)
labels = batch['labels'].to(device)


In [ ]:
'''
Trains your FusionEmotionModel using your train_loader

Computes loss and metrics

Saves a checkpoint every 5 epochs to Google Drive'''

'\nTrains your FusionEmotionModel using your train_loader\n\nComputes loss and metrics\n\nSaves a checkpoint every 5 epochs to Google Drive'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
checkpoint_dir = "/content/drive/MyDrive/emotion_model_checkpoints"
import os
os.makedirs(checkpoint_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 20

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/emotion_model_checkpoints/model_epoch_10.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1


In [ ]:
from tqdm import tqdm
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    ##for batch in train_loader: ###batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        token_ids = batch['token_ids'].to(device)
        audio = batch['audio'].to(device)
        video = batch['video'].to(device)
        labels = batch['labels'].squeeze().to(device)

        # Forward pass
        text_embed = get_text_embedding(token_ids)
        outputs = model(text_embed, audio, video)

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        all_preds.append(outputs.detach())
        all_labels.append(labels.detach())

    # Metrics
    preds = torch.cat(all_preds)
    labels = torch.cat(all_labels)
    acc, f1 = evaluate_metrics(preds, labels)

    print(f"Epoch {epoch}: Loss={total_loss:.4f}, Acc={acc:.4f}, F1={f1:.4f}")

    # Save checkpoint every 5 epochs
    if epoch % 1 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }, checkpoint_path)
        print(f"✅ Saved checkpoint to {checkpoint_path}")


Epoch 1: 100%|██████████| 2497/2497 [32:54<00:00,  1.26it/s]


Epoch 1: Loss=3787.2110, Acc=0.4716, F1=0.3030
✅ Saved checkpoint to /content/drive/MyDrive/emotion_model_checkpoints/model_epoch_1.pt


Epoch 2: 100%|██████████| 2497/2497 [34:46<00:00,  1.20it/s]


Epoch 2: Loss=3675.9479, Acc=0.4829, F1=0.3420
✅ Saved checkpoint to /content/drive/MyDrive/emotion_model_checkpoints/model_epoch_2.pt


Epoch 3: 100%|██████████| 2497/2497 [32:15<00:00,  1.29it/s]


...

Epoch 17: 100%|██████████| 2497/2497 [32:09<00:00,  1.29it/s]


Epoch 17: Loss=3154.2910, Acc=0.5678, F1=0.5073
✅ Saved checkpoint to /content/drive/MyDrive/emotion_model_checkpoints/model_epoch_17.pt


Epoch 18: 100%|██████████| 2497/2497 [32:05<00:00,  1.30it/s]


Epoch 18: Loss=3136.2223, Acc=0.5681, F1=0.5087
✅ Saved checkpoint to /content/drive/MyDrive/emotion_model_checkpoints/model_epoch_18.pt


Epoch 19:  49%|████▉     | 1226/2497 [15:45<17:52,  1.19it/s]

In [ ]:
### for get better loss
from tqdm import tqdm

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        token_ids = batch['token_ids'].to(device)
        audio = batch['audio'].to(device)
        video = batch['video'].to(device)
        labels = batch['labels'].squeeze().to(device)

        # Forward pass
        text_embed = get_text_embedding(token_ids)
        outputs = model(text_embed, audio, video)

        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_preds.append(outputs.detach())
        all_labels.append(labels.detach())

    # Normalize loss
    avg_loss = running_loss / len(train_loader)

    # Metrics
    preds = torch.cat(all_preds)
    labels = torch.cat(all_labels)
    acc, f1 = evaluate_metrics(preds, labels)

    print(f"📊 Epoch {epoch}: Avg Loss = {avg_loss:.4f}, Accuracy = {acc:.4f}, F1 Score = {f1:.4f}")

    # Save checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss
    }, checkpoint_path)
    print(f"✅ Saved checkpoint to {checkpoint_path}")


In [ ]:
## Epoch 13: Loss=3243.4623, Acc=0.5545, F1=0.4877
'''
You're building a fantastic multimodal emotion recognition pipeline using the
 MELD dataset—text, audio, and video fused with attention and GRU layers.'''

In [ ]:
total_loss = 0
for batch in dataloader:
    outputs = model(batch['input'])
    loss = criterion(outputs, batch['label'])
    total_loss += loss.item()

avg_loss = total_loss / len(dataloader)  # Normalize by number of batches
avg_loss = total_loss / total_samples
##If You're Using a Custom Loss Function

def custom_loss(pred, target):
    loss = torch.sum((pred - target)**2)
    return loss / pred.size(0)  # Normalize by batch size

In [ ]:
'''
Yes, exactly—that’s the goal of your model! 🎯

You're building a multimodal emotion recognition model that takes in:

    Text (via BERT embeddings)

    Audio features (like tone, pitch, etc.)

    Video features (like facial expressions or gestures)

…and fuses them together to predict one of several emotion classes (like happy, sad, angry, etc.).'''

In [ ]:
''' 🔥 Why This Model Works for Emotion Prediction

    Text captures what’s being said

    Audio captures how it’s being said

    Video captures non-verbal cues like facial expressions

    Attention pooling helps the model focus on the most emotionally relevant parts of audio/video sequences

    Fusion layer combines all modalities into a single representation for classification

So yes—once trained, your model will take a new sample (text + audio + video) and output a predicted emotion label.'''


In [ ]:
### multimodal emotion recognition model.
'''Parameter	Suggested Value	Why It Works
Learning Rate (lr)	1e-4 or 5e-5	Small enough for BERT fine-tuning, stable for GRUs
Epochs	15–30	Enough to converge without overfitting
Batch Size	4–16 (you’re using 4)	Small batches help generalization, especially with limited data
'''


In [ ]:
'''The good news? You’ve now got a full training loop, checkpointing, and visualization tools all wired up. Once this run finishes, you’ll have a model that not only learns but also shows you how it's learning—and saves itself along the way.

If you want to speed up future experiments, I can help you:

    Add validation and early stopping

    Automate hyperparameter sweeps

    Run faster with mixed precision or gradient accumulation'''

In [ ]:
'''✅ Additions to Your Training Loop
1. Validation Step

Evaluate model performance on a separate validation set each epoch.
2. Early Stopping

Stop training if validation loss doesn’t improve for a set number of epochs.
3. Best Model Saving

Save the model with the best validation F1 score, not just every 5 epochs.
4. Progress Bar

Use tqdm to show live progress during each epoch.'''

In [ ]:
from tqdm import tqdm
import copy

best_f1 = 0
patience = 5
patience_counter = 0

train_losses = []
val_losses = []
train_f1s = []
val_f1s = []

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in tqdm(train_loader, desc=f"Epoch {epoch} [Training]"):
        token_ids = batch['token_ids'].to(device)
        audio = batch['audio'].to(device)
        video = batch['video'].to(device)
        labels = batch['labels'].squeeze().to(device)

        text_embed = get_text_embedding(token_ids)
        outputs = model(text_embed, audio, video)

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        all_preds.append(outputs.detach())
        all_labels.append(labels.detach())

    train_preds = torch.cat(all_preds)
    train_labels = torch.cat(all_labels)
    train_acc, train_f1 = evaluate_metrics(train_preds, train_labels)

    train_losses.append(total_loss)
    train_f1s.append(train_f1)

    print(f"Epoch {epoch} — Train Loss: {total_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")

    # Validation step
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch} [Validation]"):
            token_ids = batch['token_ids'].to(device)
            audio = batch['audio'].to(device)
            video = batch['video'].to(device)
            labels = batch['labels'].squeeze().to(device)

            text_embed = get_text_embedding(token_ids)
            outputs = model(text_embed, audio, video)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            val_preds.append(outputs)
            val_labels.append(labels)

    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)
    val_acc, val_f1 = evaluate_metrics(val_preds, val_labels)

    val_losses.append(val_loss)
    val_f1s.append(val_f1)

    print(f"Epoch {epoch} — Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")

    # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_model = copy.deepcopy(model.state_dict())
        torch.save(best_model, os.path.join(checkpoint_dir, "best_model.pt"))
        print("✅ Best model saved!")

        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

    # Save checkpoint every 5 epochs
    if epoch % 5 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }, checkpoint_path)
        print(f"📦 Checkpoint saved at {checkpoint_path}")


In [ ]:
'''
save best model :
🔍 What Most Practitioners Do

    Classification with imbalance: Save based on F1 score.

    Classification with balance: Save based on accuracy or loss.

    Regression or generative tasks: Save based on validation loss.
'''

In [ ]:
''' your fusion model is designed to predict exactly those seven emotions: Anger, Disgust, Sadness, Joy, Neutral, Surprise, and Fear. That’s a classic set for emotion recognition tasks, especially in datasets like MELD, IEMOCAP, or EmoReact.

Here’s how your model is set up to handle that:
🎯 Why Your Model Is Well-Suited for This Task

    Multimodal Inputs: You're combining text, audio, and video—which is crucial because emotions are expressed not just in words, but in tone and facial expressions.

    Temporal Modeling: Using GRUs for audio and video lets your model capture how emotions evolve over time within an utterance.

    Attention Mechanism: This helps the model focus on the most emotionally relevant parts of the sequence—like a spike in vocal pitch or a sudden facial reaction.

    Fusion Layer: By combining all three modalities, your model builds a richer representation of the speaker’s emotional state.

    Output Layer: Your final layer outputs logits for 7 classes, which correspond to the emotions you listed.

🧠 A Few Things to Double-Check

    Label Encoding: Make sure your emotion labels are mapped consistently (e.g., {'Anger': 0, 'Disgust': 1, ..., 'Fear': 6}).

    Loss Function: You're using CrossEntropyLoss, which is perfect for multi-class classification.

    Evaluation: Use macro or weighted F1 score to account for class imbalance (some emotions like Disgust or Surprise might be rarer).
    '''

In [ ]:
''' Step-by-Step: Preparing and Feeding Test Data
1. Understand the Test Data Structure

Your test CSV likely includes columns like:

    Utterance: the spoken sentence (text modality)

    Speaker: who said it

    Emotion: ground truth label (for evaluation)

    Dialogue_ID, Utterance_ID: for context tracking

    Audio_Path or Video_Path: paths to audio/video files (if available)'''


In [ ]:
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# text_inputs = tokenizer(list(test_df['Utterance']), padding=True, truncation=True, return_tensors="pt")

# import torchaudio

# def extract_audio_features(path):
#     waveform, sample_rate = torchaudio.load(path)
#     # Example: MFCCs
#     mfcc = torchaudio.transforms.MFCC()(waveform)
#     return mfcc.mean(dim=-1)  # shape: [channels, features]

# audio_features = [extract_audio_features(f"/content/MELD.Raw/audio/{uid}.wav") for uid in test_df['Utterance_ID']]
# Assuming precomputed video embeddings

## If you're using the preprocessed features from test.pkl, then you do not need to manually extract audio features from MELD.Raw/audio/. The .pkl files already contain aligned and pre-extracted features for each modality: text, audio, and video.
##video_features = torch.load("video_test_embeddings.pt")  # shape: [num_samples, feature_dim]


In [ ]:
import pickle

with open('test.pkl', 'rb') as f:
    test_features = pickle.load(f)

test_df['key'] = test_df['Dialogue_ID'].astype(str) + '-' + test_df['Utterance_ID'].astype(str)

text_feats = []
audio_feats = []
video_feats = []
labels = []

for key in test_df['key']:
    sample = test_features.get(key)
    if sample:
        text_feats.append(sample['text'])     # shape: [768] or similar
        audio_feats.append(sample['audio'])   # shape: [n_audio_features]
        video_feats.append(sample['video'])   # shape: [n_video_features]
        labels.append(sample['label'])        # optional, for evaluation
import torch

text_tensor = torch.tensor(text_feats)
audio_tensor = torch.tensor(audio_feats)
video_tensor = torch.tensor(video_feats)

model.eval()
with torch.no_grad():
    logits = model(text_tensor, audio_tensor, video_tensor)
    predictions = torch.argmax(logits, dim=1)

emotion_map = {0: 'Anger', 1: 'Disgust', 2: 'Sadness', 3: 'Joy', 4: 'Neutral', 5: 'Surprise', 6: 'Fear'}
predicted_emotions = [emotion_map[p.item()] for p in predictions]

from sklearn.metrics import classification_report

print(classification_report(labels, predicted_emotions))

In [ ]:
### You’re not lying by saying you built a project with the help of tools. You’re being resourceful. And that’s exactly what good engineers do.
''' You understand the architecture, the data flow, and the purpose of each component.

You can explain your choices: why you used certain loss functions, how you handled class imbalance, etc.

You’ve experimented, tweaked, and learned from the process.'''
